### Movie Recommendations with Association Rules

Download small dataset : https://grouplens.org/datasets/movielens/ (or from included zip file)


In [4]:
###1. Load data 
import pandas as pd
ratings_df = pd.read_csv("ml-latest-small/ratings.csv",delimiter=",")
movies_df = pd.read_csv("ml-latest-small/movies.csv",delimiter=",")
#all_ratings["timestamp"] = pd.to_datetime(all_ratings['timestamp'])
print(ratings_df.head())

   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205


In [5]:
movie_ratings_df = pd.merge(ratings_df, movies_df, how='left',
                            left_on='movieId', right_on='movieId')

In [6]:
movie_ratings_df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


Create a Pivot table using the panda function pivot

In [7]:
rat=ratings_df.pivot(index='userId', columns='movieId', values='rating')
rat.fillna(0,inplace=True) #replace NaN with 0
rat.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
sel=[31,1029,1061,1129,1172] #movieId
#the ratings the above movieId for userid = 1 [check that it is the same as above ]
rat.loc[1][sel] 

movieId
31      2.5
1029    3.0
1061    3.0
1129    2.0
1172    4.0
Name: 1, dtype: float64

In [9]:
rat[rat<3]=0
rat[rat>=3]=1 #Good rating

In [10]:
rat.loc[1][sel] 

movieId
31      0.0
1029    1.0
1061    1.0
1129    0.0
1172    1.0
Name: 1, dtype: float64

In [11]:
rat.sum(axis=1) # how many ratings>=3 for each user 

userId
1        8.0
2       70.0
3       47.0
4      194.0
5       96.0
       ...  
667     64.0
668     18.0
669     30.0
670     26.0
671    106.0
Length: 671, dtype: float64

### Your turn
1. Create a frequent_itemsets with support of 0.2 and set use_colnames=True
2. Create the association rules with apriori with confidence metric = 0.5

In [12]:
#Q1
from mlxtend.frequent_patterns import apriori

frequent_itemsets=apriori(rat, min_support=0.2,use_colnames=True) #run the association rule algorithm
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.332340,(1),1
1,0.277198,(32),1
2,0.277198,(47),1
3,0.293592,(50),1
4,0.309985,(110),1
...,...,...,...
130,0.226528,"(296, 356, 318)",3
131,0.235469,"(296, 593, 318)",3
132,0.202683,"(296, 480, 356)",3
133,0.241431,"(296, 593, 356)",3


In [13]:
#Q2
from mlxtend.frequent_patterns import association_rules

association_rules=association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
association_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1),(260),0.332340,0.406855,0.207154,0.623318,1.532039,0.071939,1.574658
1,(260),(1),0.406855,0.332340,0.207154,0.509158,1.532039,0.071939,1.360233
2,(1),(296),0.332340,0.469449,0.202683,0.609865,1.299110,0.046666,1.359919
3,(1),(356),0.332340,0.476900,0.211624,0.636771,1.335230,0.053132,1.440139
4,(296),(47),0.469449,0.277198,0.238450,0.507937,1.832395,0.108320,1.468920
...,...,...,...,...,...,...,...,...,...
156,(356),"(296, 593)",0.476900,0.320417,0.241431,0.506250,1.579971,0.088624,1.376370
157,"(593, 356)",(318),0.299553,0.454545,0.219076,0.731343,1.608955,0.082916,2.030303
158,"(593, 318)",(356),0.292101,0.476900,0.219076,0.750000,1.572656,0.079773,2.092399
159,"(356, 318)",(593),0.295082,0.429210,0.219076,0.742424,1.729745,0.092424,2.216008


### Questions
1. If a person likes The Matrix, and Star Wars, what other movies would you recommend to him/her? Why?
1. If a person likes Pulp Fiction and Jurassic Park, what other movies would you recommend to him/her? Why?
1. Comment on line 147 and 148, on why is there a difference in values for conviction and confidence:

147 	(296, 480) 	(356) 	0.244411 	0.476900 	0.202683 	0.829268 	1.738872 	0.086123 	3.063871

148 	(296, 356) 	(480) 	0.311475 	0.366617 	0.202683 	0.650718 	1.774925 	0.088490 	1.813384

Reference: http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/#metrics



In [14]:
#Q1
#Matrix(1999) ID is 2571
#Star Wars:Episode IV- A New Hope (1977) ID is 260
#Movie ID 1196 is Star Wars: Episode V - The Empire Strikes Back (1980)
#recommend Star Wars: Episode V - The Empire Strikes Back (1980) to him/her
#The reason is because there is confidence of up to 0.84472

association_rules[association_rules['antecedents'] == {2571,260}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
135,"(2571, 260)",(1196),0.23994,0.33234,0.202683,0.84472,2.541737,0.122941,4.299732


In [15]:
#Q2
#Pulp Fiction(1994) ID is 296
#Jurassic Park(1993) ID is 480
#Movie ID 356 is Forrest Gump (1994)
#recommend Forrest Gump (1994) to him/her
#The reason is because there is confidence of up to 0.829268

association_rules[association_rules['antecedents'] == {296,480}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
147,"(296, 480)",(356),0.244411,0.4769,0.202683,0.829268,1.738872,0.086123,3.063871


In [16]:
#Q3
#Because There is no association between movie ID 480(Jurassic Park) and movie ID 356(Forrest Gump).
association_rules[147:149]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
147,"(296, 480)",(356),0.244411,0.476900,0.202683,0.829268,1.738872,0.086123,3.063871
148,"(296, 356)",(480),0.311475,0.366617,0.202683,0.650718,1.774925,0.088490,1.813384
